In [1]:
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess
from monai.networks.blocks import Warp
import torch
import pandas as pd

In [6]:
import argparse
import logging
import sys

import monai
import torch
from torch.utils.tensorboard import SummaryWriter
from monai.utils import set_determinism
import numpy as np

from seg_model import getUNetForSegmentation, getUNETRForSegmentation
from transforms_dict import getSegmentationPostProcessingForLabel, getSegmentationPostProcessingForLabelOutput, getSegmentationPostProcessingForAllLabelsOutput
from utils import compute_mean_dice, getReducePlateauScheduler, getAdamOptimizer, loadExistingModel
from utils import print_model_output, check_model_name, getDevice
from monai.transforms import Activations, EnsureType, Compose
from torch import nn
from transforms_dict import getSegmentationEvalTransformsForMRI, getSegmentationPostProcessingForMaskOutput
from transforms_dict import getSegmentationInverseTransformForLabels, SaveTransformForMRI

In [9]:
model = getUNetForSegmentation()
modelname = "test_labels.pth"

modelname = check_model_name(modelname)
set_determinism(seed=0)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
device = getDevice()

In [44]:
device = getDevice()
outputs_processing = getSegmentationPostProcessingForLabelOutput(axis=1)
all_outputs_processing = getSegmentationPostProcessingForAllLabelsOutput(axis=1)
labels_processing = getSegmentationPostProcessingForLabel()
test_proba = Compose([
    EnsureType(),
    Activations(other=nn.Softmax(dim=1)),
    #GetMaxChannelWise(),
    EnsureType(data_type="tensor", device=device)
])

#activations
softmax = Activations(other=nn.Softmax(dim=1))

In [54]:
N4 = False
resample = False
save_tmp = False
get_largest_component = False

loadExistingModel(model, None, ft=modelname)
model.eval()

mris = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "*_id.nii.gz")))

for mriname in mris:
    outname = "tmp.nii.gz"
    mri_preprocessing = getSegmentationEvalTransformsForMRI(N4=N4, outname=outname, save=save_tmp)
    with torch.no_grad():
        inputs = mri_preprocessing(mriname).to(device)
        outputs = model(inputs).cpu()  
        preds = outputs_processing(outputs)
        all_preds = all_outputs_processing(outputs)
        probs = test_proba(outputs)
        
        all_preds = all_preds.unsqueeze(0)
        xdptdr = nib.load(mriname)
        mask = nib.Nifti1Image(all_preds.cpu().numpy().squeeze(), xdptdr.affine, xdptdr.header)
        print(mriname)
        nib.save(mask,"test.nii.gz")
        
    break

dataset3/GIN_Mean/MRI/653_N4_o_128_ras_norm_id.nii.gz


In [48]:
print(outputs.shape)
print(all_preds.shape)
print('-'*5)

w = 70
print(outputs[0,:,w,w,w].cpu().numpy())
print(preds[0,:,w,w,w].cpu().numpy())
print(all_preds[0,w,w,w].item())
print(probs[0,:,w,w,w].cpu().numpy())



(1, 4, 128, 128, 128)
(1, 128, 128, 128)
-----
[ 8.846987   1.0409136 -2.4249673 -3.243588 ]
[1. 0. 0. 0.]
0
[9.9957460e-01 4.0708086e-04 1.2719439e-05 5.6097761e-06]
